In [3]:
# !pip install optuna -q
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["figure.dpi"] = 100

# Training the final model with all training data

In [7]:
file_path = 'training_data.csv'
data = pd.read_csv(file_path)

data['increase_stock_binary'] = data['increase_stock'].map(lambda x: 0 if x == 'low_bike_demand' else 1)
data = data.drop(['snow', 'increase_stock'], axis=1)
# pick out the labels
y = data['increase_stock_binary'].to_numpy().astype(np.float32)
y = torch.tensor(y).unsqueeze(-1)
data = data.drop(['increase_stock_binary'], axis=1)


# Separating the numerical and categorical data to handle them separately
onehot = True
if onehot:
    cat_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == int})
    num_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == float})
    
    # performing onehot encoding on the categorical data
    cat_data = torch.from_numpy(cat_data.to_numpy())
    cat_onehot = torch.cat([F.one_hot(x, num_classes=24) for x in cat_data]).view(cat_data.shape[0], -1)
    
    # constructing the complete input dataset
    data = np.concatenate([num_data.to_numpy(), cat_onehot.numpy()], axis=-1).astype(np.float32)

# scaling
X = MinMaxScaler().fit_transform(data) 
X = torch.Tensor(X)

#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=0)
X_train = X
y_train = y

In [8]:
class DNN(nn.Module):
    def __init__(self, hidden_size, input_size, layers=2, seed=0, submodule=False):
        super().__init__()
        torch.manual_seed(seed)
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.submodule = submodule
        
        self.activation = nn.ReLU()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.batch_norm_input = nn.BatchNorm1d(hidden_size)  # BatchNorm for input layer
        self.linear_layers = nn.ModuleList()
        self.batch_norm_layers = nn.ModuleList()  # BatchNorm for other layers

        for _ in range(layers):
            self.linear_layers.append(nn.Linear(hidden_size, hidden_size))
            self.batch_norm_layers.append(nn.BatchNorm1d(hidden_size))

        self.output_layer = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.batch_norm_input(x)
        x = self.activation(x)

        for layer, batch_norm in zip(self.linear_layers, self.batch_norm_layers):
            residual = x
            x = layer(x)
            x = batch_norm(x)  
            x = self.activation(x + residual)
        
        if not self.submodule:
            x = self.output_layer(x)
            x = torch.sigmoid(x) 
        return x

In [51]:
epochs = 100 
loss_fn = nn.BCELoss()
n_ensembles = 10 # checking the performance for different weight initialization
kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
optimal_models = []

for n in range(n_ensembles):
    for cv, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        model = DNN(200, X.shape[-1], 2, seed=n)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1)
        
        train_loss_history = []
        
        for epoch in tqdm(range(epochs)):
            optimizer.zero_grad()
            model.train()
            y_pred = model(X_train[train_index])
            loss = loss_fn(y_pred, y_train[train_index])
            loss.backward()
            optimizer.step()
            train_loss_history.append(loss.item())
            
            if loss.item() <= min(train_loss_history):
                optimal_weights = model.state_dict()
                
        optimal_models.append(optimal_weights)  

100%|██████████| 100/100 [00:01<00:00, 54.24it/s]


# Testing

In [54]:
file_path = 'test_data.csv'
data = pd.read_csv(file_path)
data = data.drop(['snow'], axis=1)


# Separating the numerical and categorical data to handle them separately
onehot = True
if onehot:
    cat_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == int})
    num_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == float})
    
    # performing onehot encoding on the categorical data
    cat_data = torch.from_numpy(cat_data.to_numpy())
    cat_onehot = torch.cat([F.one_hot(x, num_classes=24) for x in cat_data]).view(cat_data.shape[0], -1)
    
    # constructing the complete input dataset
    data = np.concatenate([num_data.to_numpy(), cat_onehot.numpy()], axis=-1).astype(np.float32)

# scaling
X = MinMaxScaler().fit_transform(data) 
X_test = torch.Tensor(X)

In [55]:
def DeepNeuralNetworkEnsemble(optimal_models, test_data):
    DNN_ensemble = []
    for weights in optimal_models:
        model = DNN(200, X.shape[-1], 2)
        model.load_state_dict(weights)
        y_pred = model(test_data)
        DNN_ensemble.append(y_pred.detach().numpy())
    return torch.Tensor(DNN_ensemble).mean(0).round()
    
prediction = DeepNeuralNetworkEnsemble(optimal_models, X_test)

In [65]:
np.savetxt("DNN test prediction.csv", prediction.detach().numpy().flatten().astype(int)), delimiter=",")

ValueError: Expected 1D or 2D array, got 0D array instead

',0\n0,0\n1,1\n2,1\n3,0\n4,0\n5,0\n6,1\n7,0\n8,0\n9,0\n10,0\n11,1\n12,0\n13,0\n14,0\n15,1\n16,0\n17,0\n18,0\n19,0\n20,0\n21,0\n22,0\n23,1\n24,0\n25,0\n26,0\n27,0\n28,1\n29,0\n30,0\n31,0\n32,0\n33,0\n34,0\n35,0\n36,0\n37,0\n38,1\n39,0\n40,0\n41,1\n42,0\n43,0\n44,1\n45,0\n46,0\n47,1\n48,1\n49,0\n50,0\n51,1\n52,0\n53,0\n54,0\n55,0\n56,1\n57,0\n58,0\n59,1\n60,0\n61,0\n62,0\n63,0\n64,1\n65,0\n66,0\n67,0\n68,0\n69,0\n70,0\n71,0\n72,1\n73,0\n74,0\n75,0\n76,0\n77,0\n78,0\n79,0\n80,0\n81,0\n82,0\n83,1\n84,0\n85,0\n86,0\n87,0\n88,0\n89,1\n90,0\n91,0\n92,0\n93,0\n94,0\n95,0\n96,0\n97,0\n98,0\n99,0\n100,0\n101,0\n102,1\n103,0\n104,1\n105,1\n106,1\n107,0\n108,0\n109,0\n110,1\n111,0\n112,0\n113,0\n114,0\n115,1\n116,0\n117,0\n118,0\n119,0\n120,0\n121,0\n122,0\n123,1\n124,0\n125,0\n126,0\n127,0\n128,1\n129,0\n130,1\n131,0\n132,0\n133,0\n134,1\n135,0\n136,0\n137,0\n138,0\n139,0\n140,0\n141,0\n142,0\n143,0\n144,0\n145,0\n146,1\n147,0\n148,0\n149,0\n150,0\n151,0\n152,0\n153,0\n154,0\n155,1\n156,1\n157,0\